In [1]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import pandas as pd

In [2]:
driver = webdriver.Chrome()

In [17]:
def get_raw_data(url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "headerLeague__wrapper")))

    while True:
        # Find all current "Show more" links
        show_more_links = driver.find_elements(By.XPATH, "//span[contains(text(), 'Show more matches')]/parent::a")
        if not show_more_links:
            break  # No more links to click

        for link in show_more_links:
            try:
                # Scroll to the link to ensure it's visible
                driver.execute_script("arguments[0].scrollIntoView();", link)
                link.click()
                # Wait for the link to become stale (content expanded)
                WebDriverWait(driver, 5).until(EC.staleness_of(link))
            except Exception:
                pass  # Ignore if already expanded or error

    soup = bs(driver.page_source, 'html.parser')
    return soup

In [18]:
soup = get_raw_data('https://www.livesport.com/uk/football/england/premier-league-2023-2024/results/')

In [25]:
events = soup.find_all('div', class_='event__match event__match--withRowLink event__match--static event__match--twoLine')
last_event = soup.find_all('div', class_='event__match event__match--withRowLink event__match--static event__match--last event__match--twoLine')

In [23]:
events[-1].find('div', class_='event__time')

<div class="event__time">12.08. 12:30</div>

In [29]:
len(last_event)

44

* The match fixtures are structured in rounds, as if they were individual tables separating the first to the last event of each round
* You can use the events variable to get the regular events and the last_event to get the last event of each round respectively
* However, despite the data exploration, there seems to be a mismatch between the length of last event and number of rounds. Which is left to be solved
* Nonetheless, some progress was made.

# <h4>Fix the `last_event` length mismatch and craft a function to fully retrieve data</h4>

In [31]:
last_event[-1].find('div', class_='event__time')

<div class="event__time">11.08. 20:00</div>